## Data Preparation

This notebook is used to clean up, preprocess, and aggregate data necessary for STXGB models.

Please refer to the `Method` section of the article and Supplementary Information document for more information.


In order to run this notebook, you have to download the required datasets from this [address](https://drive.google.com/drive/u/1/folders/1laAZFCvsPLLaKDvg0isTMMr20kMe0x_r). Once downloaded, set the directory in which you have save the files as `input_directory` in the cell below.

In [36]:
input_directory = '/home/dante/SpatialData/spatial_project/data/'

In [37]:
import pandas as pd
import numpy as np
import geopandas as gpd
import platform
from datetime import datetime, timedelta

import requests
from urllib.request import urlopen
import json
import os
from os import listdir

#viz
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker



pd.set_option('display.max_rows', 5)
pd.set_option(‘mode.chained_assignment’, None) # need to be careful with this one!

SyntaxError: invalid character in identifier (2783386656.py, line 22)

In [ ]:
def datetimer(cellstring):
    if cellstring == "Total":
        return cellstring
    strlist = cellstring.split(" ")
    return f"{strlist[1]}-{strlist[3]}"

In [ ]:
datetimer("Year 2020 Week 01")

In [ ]:
#Set date string formatting based on operating system

if platform.system() == 'Windows':
    conversion_format = '%#m/%#d/%y'
else:
    conversion_format = '%-m/%-d/%y'

## Preparing hospital data by province

In [ ]:
# Taking the data straight from a csv from their pivot webpage. Address found in the good_resources.txt file.
df_cov = pd.read_csv('/home/dante/SpatialData/spatial_project/data/covid/fact_epirapo_covid19case.csv',sep=";")
mun_prov = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/municipality_province_map.csv',encoding = "ISO-8859-1",sep=";",header=1)
prov_lang_map = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/mob_map_nuts3.txt').iloc[:,1:]

In [38]:
# Making actual mappings (municipalities)
mun_prov_map = dict(zip(mun_prov.iloc[:,1],mun_prov.iloc[:,3]))
muns = list(mun_prov.iloc[:,1])

# Making actual mappings (provinces)
prov_lang_map = prov_lang_map[prov_lang_map.iloc[:,3] != 'Åland']
prov_fin_eng = dict(zip(prov_lang_map.iloc[:,1],prov_lang_map.iloc[:,3]))
provs = list(set(mun_prov.iloc[:,3]))

In [39]:
# Dropping everything but the municipalities (contains data about hospital district as well)
df_cov = df_cov[df_cov.Alue.isin(muns)]

# Dropping "kaikki ajat" aka "all times":
df_cov = df_cov[df_cov['Aika'] != 'Kaikki ajat']

# Mapping municipalities to the correct province
df_cov['province'] = df_cov.iloc[:,1].map(mun_prov_map)

# Dropping Ahvenanmaa (Åland) - the autonomous island we don't 
# analyze in this thing
df_cov = df_cov[df_cov.province != 'Ahvenanmaa']

# Resetting index after all this
df_cov.reset_index(drop=True,inplace=True)

# Setting val as int
df_cov['val'] = np.where(df_cov['val'] == '..',0,df_cov['val'])
df_cov['val'] = df_cov.val.astype(int)

# Grouping by both province and week
df_cov = df_cov.groupby(['province','Aika']).agg({'val' : 'sum'}).reset_index(drop=False)

# Pivoting, so that we get cases along dimensions (time x province)
df_cov_pivot = df_cov.pivot('Aika','province','val').reset_index(drop=False).rename_axis(None, axis=1)

# Putting it in the year-week format according to the rest of the data
df_cov_pivot['Y-W'] = df_cov_pivot.Aika.apply(datetimer)
df_cov_pivot.drop(columns=['Aika'],inplace=True)

# Data seems uniform - just how we like it!
print(df_cov.province.value_counts())

# Changing province names into English:
df_cov_pivot.rename(columns=prov_fin_eng,inplace=True)

AttributeError: 'DataFrame' object has no attribute 'Alue'

In [ ]:
df_cov_pivot.head(10)
# df_cov_pivot.to_csv('/home/dante/SpatialData/spatial_project/data/processed/newcases_by_week_province.csv',index=False,)

In [ ]:
df_cov_pivot.iloc[:,0:-1] = df_cov_pivot.iloc[:,0:-1].apply(lambda x: np.log(x+1))

In [ ]:
df_cov_pivot

#### Visualizing the measure

In [ ]:
f = plt.figure()
f.set_figheight(9)
f.set_figwidth(15)

ax1 = f.add_subplot(111)



sns.lineplot(x='Y-W',
             y='value',
             hue = 'variable',
             data=pd.melt(df_cov_pivot, ['Y-W']),
             ax=ax1)

fontdict = {'fontsize': 20,
#  'fontweight' : rcParams['axes.titleweight'],
#  'verticalalignment': 'baseline',
#  'horizontalalignment': loc}
           }
plt.xlim(0,115)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax1.xaxis.set_major_formatter(ticker.ScalarFormatter())
ax1.set_title(r"Ln new cases (by Week x Province)",fontdict = fontdict)
ax1.set_xlabel(r"Week number (starting from 2020-01)",fontsize=14)
ax1.set_ylabel(r"Ln new cases",rotation=0,labelpad=30,fontsize=14)



leg = plt.legend(title='Province', loc='upper left', labels=list(df_cov_pivot.columns[0:-1]))
# leg._legend_box.align = "top"
plt.setp(leg.get_title(),fontsize='large')
plt.savefig('/home/dante/SpatialData/spatial_project/data/processed/new_cases_timeseries.png')
plt.show()

## Google intra-region mobility data

### Script that combines data from all years

In [ ]:
# Specifying the path to the folder with google raw data
path = '/home/dante/SpatialData/spatial_project/data/google/'

# Getting the name of all files (2020-2022)
filelist = [file for file in os.listdir(path) if '.csv' in file]

In [ ]:
# Specifying dtypes for faster processing of csv...
dtypes = {
    'country_region_code' : 'object',
    'country_region' : 'object',
    'sub_region_1' : 'object',
    'sub_region_2' : 'object',
    'metro_area' : 'object',
    'iso_3166_2_code' : 'object',
    'census_fips_code' : 'object',
    'place_id' : 'object',
    'date' : 'object',
    'retail_and_recreation_percent_change_from_baseline' : 'float64',
    'grocery_and_pharmacy_percent_change_from_baseline' : 'float64',
    'parks_percent_change_from_baseline' : 'float64',
    'transit_stations_percent_change_from_baseline' : 'float64',
    'workplaces_percent_change_from_baseline' : 'float64',
    'residential_percent_change_from_baseline' : 'float64',
}

# Assigning the correct column names
headers = list(dtypes.keys())

# Telling Pandas what column is a datecolumn (so that it can be parsed)
parse_dates = ['date']

# Creating a map between the ISO-codes of the different provinces and their ENG/FI/SWE 
# names, for legibility later on.
df_codemap = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/mob_map_nuts3.txt',sep=',')
iso_name_map = dict(zip(df_codemap.iloc[:,0],df_codemap.iloc[:,3]))

In [ ]:
df_codemap

In [ ]:
# Specifying empty dataframe for filling
df_list = []

for filename in filelist:
    # Specifying file location
    floc = path + filename
    
    # Reading in file
    df_mr = pd.read_csv(floc,header=0,dtype=dtypes,names=headers,parse_dates=parse_dates)
    
    # Sorting values by date and sub_region_2,sub_region_1 as we want to be able to
    # look at the data in an intuitive way.
    df_mr.sort_values(by=['date','sub_region_2','sub_region_1'],inplace=True)
    
    df_mr['province'] = df_mr.iso_3166_2_code.map(iso_name_map)
    
    # Dropping unneccesary columns
    df_mr.drop(columns=['country_region_code','country_region','census_fips_code','iso_3166_2_code','place_id','metro_area','sub_region_2','sub_region_1'],inplace=True)

    # Specifying down our data to the province level
    df_mr.dropna(subset=['province'],inplace=True)
    df_mr.reset_index(drop=True,inplace=True)

    # Casting date in correct format (apparently Pandas doesn't actually read the dtypes correctly)
    df_mr['date'] = pd.to_datetime(df_mr.date)

    # Adding day of year for gap checking if needed
    df_mr['dayofyear'] = df_mr['date'].dt.dayofyear

    # Defining the movement data columns of interest
    # and filling in NaNs with 0 (i.e no change/information).
    # I do not know how sustainable this assumption is...
    mvcols = [col for col in df_mr.columns if 'baseline' in col]
    values = {col : 0 for col in mvcols}
    df_mr.fillna(value=values, inplace=True)
    
    
    for prov in iso_name_map.values():
    
        # Specify the correct province 
        df = df_mr[df_mr.province == prov]

        # The below code bins the data according to week commencing MONDAY
        # This code looks complicated, but achieves something quite understanable.
        df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
        df = df.groupby([pd.Grouper(key='date', freq='W-MON')])[mvcols].mean().reset_index().sort_values('date')

        # Creating the column with which we can then merge to the cases data..
        df['Y-W'] = df.date.dt.year.astype(str)+'-'+df.date.dt.isocalendar().week.astype(str)

        # Adding name of province
        df['province'] = prov
        df.reset_index(drop=True) 

        # Finally, appending to the list of dfs...
        df_list.append(df)
    
# Finally, concatenating the whole list
combined_mv_df = pd.concat(df_list)

In [ ]:
for idx,prov in enumerate(combined_mv_df.province.value_counts().index):
    datas = combined_mv_df[combined_mv_df.province == prov].shape
    print(f"Province {idx}, shape : {datas}")

In [ ]:
# Saving the file as a CSV.
# combined_mv_df.to_csv('/home/dante/SpatialData/spatial_project/data/processed/google_mobility_by_week_province.csv',index=False)

In [ ]:
combined_mv_df.sort_values(by=['province','date'],inplace=True)
# combined_mv_df[combined_mv_df.province == 'Uusimaa'].head(120)

## FB inter-region connectivity data

### Creating the Social Proximity to Cases Variable

In [ ]:
df_sci = pd.read_csv('/home/dante/SpatialData/spatial_project/data/processed/fb_sci_by_province.csv')
df_cov = pd.read_csv('/home/dante/SpatialData/spatial_project/data/processed/newcases_by_week_province.csv')
prov_nam_conv = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/mob_map_nuts3.txt')
# mun_prov = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/municipality_province_map.csv',encoding = "ISO-8859-1",sep=";",header=1)
mun_pop = pd.read_csv('/home/dante/SpatialData/spatial_project/data/municipal/municipal_population.txt',header=0,encoding = "UTF-8",sep="\t")

In [ ]:
# Collecting provincial information
prov_pop = mun_pop.iloc[310:328,:]

# Casting the population figures as INT
prov_pop.iloc[:,1] = prov_pop.iloc[:,1].str.replace(",","").astype(int)
prov_pop.reset_index(drop=True)

# Sorting values for easier legibility
prov_pop.sort_values(by=prov_pop.columns[0],inplace=True)

prov_pop['province'] = prov_pop['Population'].map(dict(zip(prov_nam_conv.iloc[:,-2],prov_nam_conv.iloc[:,3])))

# Making province : population dict for division later...
prov_pop_dict = dict(zip(prov_pop.iloc[:,2],prov_pop.iloc[:,1]))

In [ ]:
# Below, you'll see the dict with province x population
prov_pop_dict

In [ ]:
# Changing names from THL convention to the convention we have been using.
df_cov.rename(columns=dict(zip(prov_nam_conv.iloc[:,-1],prov_nam_conv.iloc[:,-4])),inplace=True)

In [ ]:
# Having a peek at our data...
df_cov.head(10)

In [ ]:
# Creating a copy of the COVID df so that we can always restart
# the modifications from here.
df_cov_c = df_cov.copy()

# The below for-loop scales the covid cases for each day x province 
# to the level of new cases per 10k people. In this way, we are stand-
# ardizing the measurement across provinces.
for idx,prov in enumerate(df_cov.columns[0:-1]):
    #FOR EACH PROVINCE

    # For each province, fetch the population of the province,
    # and then divide the whole time series for that province
    # by the population.
    case_by_pop = df_cov_c[prov] / prov_pop_dict[prov]
    
    # Multiply the per capita number by 10000, to get cases
    # per 10k people.
    case_by_10k = case_by_pop * 10000
    
    # Assign back to the dataframe.
    df_cov_c[prov] = case_by_10k

In [ ]:
df_cov_c

In [ ]:
df_sci.head(20)

In [ ]:
# Mapping GADM NUTS3 code to our used standard for the province name
mob_map = dict(zip(prov_nam_conv.iloc[:,4],prov_nam_conv.iloc[:,3]))

# Creating a map between SCI weights of a given province and all OTHERS.
sci_dict = {}
for prov in df_sci.user_loc_province.value_counts().index:
    # FOR EACH PROVINCE
    
    # Get list of all other provinces, i.e. not the one that is being
    # considered in the loop (because this was already deleted earlier
    # when I made the fb_sci_by_province.csv file).
    fr_list = list(df_sci[(df_sci.user_loc_province == prov)].fr_loc_province)
    
    # Get the list of all other province weights.
    sci_weights_list = list(df_sci[(df_sci.user_loc_province == prov)].sci_weights)
    
    # Assign the list of all other province weights to a dictionary, with the name
    # of the 'parent' province.
    sci_dict[prov] = {fr_loc:sci_weight for fr_loc,sci_weight in zip(fr_list,sci_weights_list)}


# The SCI-dict looks as follows:
# sci_dict

In [ ]:
# FINALLY, we can create the SCI-weighted data!

# Creating a final dataframe to write to.
df_spc = df_cov.copy()

for prov in sci_dict.keys():
    #FOR PROVINCE IN THE SCI-DICT
    
    # Creating a local copy for computation
    df = df_cov_c.copy()
    
    # Get the weights for the province
    weights = sci_dict[prov]
    
    # Then, we weigh the local copy of the dataframe
    # by the weights in the sci_dict
    for neigh_prov in weights.keys():  
        #FOR EACH PROVINCE APART FROM prov
        
        # Weighting the observations by SCI weights
        df[neigh_prov] = df[neigh_prov]*weights[neigh_prov]
    
    # Collecting the names of the neighbors
    neighs = list(weights.keys())
    
    # Summing the weighted observations across all provinces 
    # not equal to the province we are processing the data for.
    df_spc[prov] = df[neighs].sum(axis=1)
    
    # Setting the data to the correct format (i.e ln(x+1)) so
    # that we won't have minus-valued predicted cases.
    df_spc[prov] = np.log(df_spc[prov]+1)

In [ ]:
f = plt.figure()
f.set_figheight(9)
f.set_figwidth(15)

ax1 = f.add_subplot(111)



sns.lineplot(x='Y-W',
             y='value',
             hue = 'variable',
             data=pd.melt(df_spc, ['Y-W']),
             ax=ax1)

fontdict = {'fontsize': 20,
#  'fontweight' : rcParams['axes.titleweight'],
#  'verticalalignment': 'baseline',
#  'horizontalalignment': loc}
           }
plt.xlim(0,115)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax1.xaxis.set_major_formatter(ticker.ScalarFormatter())
ax1.set_title(r"Social Proximity to Cases (SPC))",fontdict = fontdict)
ax1.set_xlabel(r"Week number (starting from 2020-01)",fontsize=14)
ax1.set_ylabel(r"$SPC_{i,t}$",rotation=0,labelpad=30,fontsize=14)



leg = plt.legend(title='Province', loc='upper left', labels=list(df_spc.columns[0:-1]))
# leg._legend_box.align = "top"
plt.setp(leg.get_title(),fontsize='large')
plt.savefig('/home/dante/SpatialData/spatial_project/data/processed/SPC_timeseries.png')
plt.show()

- I have a feeling that these are always driven by Uusimaa..

In [ ]:
# This now holds the SCI weighted cases!    
df_spc.head(15)

# df_spc.to_csv('/home/dante/SpatialData/spatial_project/data/processed/spc_by_week_province.csv',index=False)

## Weather data
- The weather data was gathered from the Finnish Meteorological Institute (FMI)
https://en.ilmatieteenlaitos.fi/download-observations.

In [ ]:
filepath = '/home/dante/SpatialData/spatial_project/data/weather/'
filelist = [file for file in os.listdir(filepath) if '.csv' in file]
provinces = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/mob_map_nuts3.txt',sep=',').iloc[:,3].values.tolist()

In [ ]:
# Initializing an empty list to fill with weather data
df_list = []

for idx,prov in enumerate(provinces):
    # FOR EACH PROVINCE NAME
    
    # Create filename
    path2data = filepath + str(prov) + '.csv'
    
    # Fetch data, parse it
    df = pd.read_csv(path2data,
                     parse_dates = {"date" : ["Year","m","d"]},
                     dtype = {'Time zone' : 'object',
                              'Precipitation amount (mm)' : 'float64',
                              'Maximum temperature (degC)' : 'float64',
                              'Minimum temperature (degC)' : 'float64'},
#                      nrows=1,
                     header=0)
    
    print(df.shape)
    # Assigning columns of interest
    cols = list(df.columns)[2:]
        
    # The below code bins the data according to week commencing MONDAY
    # This code looks complicated, but achieves something quite understanable.
    df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
    df = df.groupby([pd.Grouper(key='date', freq='W-MON')])[cols].mean().reset_index().sort_values('date')

    # Creating the column with which we can then merge to the cases data..
    df['Y-W'] = df.date.dt.year.astype(str)+'-'+df.date.dt.isocalendar().week.astype(str)
    
    if idx == 0:
        y_w = df['Y-W']
    
    # Adding name of province
    df['province'] = prov
    df.reset_index(drop=True) 
    
    # Setting first week of January to start on 2020-01-01
    # as per data from THL (covid source)
    df.iloc[0,-2] = '2020-1'

    # Finally, appending to the list of dfs...
    df_list.append(df)
    


# concatenating it all into one big df...
weather_data = pd.concat(df_list)

In [ ]:
# weather_data.pivot('date','province','Maximum temperature (degC)').head(150).reset_index().isna().sum()
# weather_data.pivot('date','province','Minimum temperature (degC)').head(150).reset_index().isna().sum()

In [ ]:
# Getting the max temperature reading per weep for all provinces
max_temp_data = weather_data.pivot('date','province','Maximum temperature (degC)')
max_temp_data['Y-W'] = y_w.values
max_temp_data.reset_index(drop=False)

# Getting the min temperature reading per weep for all provinces
min_temp_data = weather_data.pivot('date','province','Minimum temperature (degC)')
min_temp_data['Y-W'] = y_w.values
min_temp_data.reset_index(drop=False)

In [ ]:
# min_temp_data.to_csv('/home/dante/SpatialData/spatial_project/data/processed/min_temp_by_week_province.csv',index=False)
# max_temp_data.to_csv('/home/dante/SpatialData/spatial_project/data/processed/max_temp_by_week_province.csv',index=False)

In [ ]:
data = gpd.read_file( input_directory + 'Contiguous_US.geojson')

# Or alternatively:
# url='https://drive.google.com/file/d/1MVyLzzHl3hzno4o1rLZtI0peqIi23zsr/view?usp=sharing'
# url_counties='https://drive.google.com/uc?id=' + url.split('/')[-2]
# data = gpd.read_file(url_counties)

In [ ]:
data['STATEFP'] = data.apply(lambda L: L.GEOID[:2], axis=1)

In [ ]:
global number_counties 
number_counties = data.shape[0] #3103

In [ ]:
data.sort_values(by='GEOID', inplace=True)

### Load COVID data and apply smoothing 

To alleviate inconsistencies in reporting COVID-19 cases, we apply a 7-day moving average to the case data published by JHU.

In [ ]:
def get_JH_covid_data(target, smooth):
    
    '''
    Parameters:
    --------------
        target: str
            the target variable: either 'case' or 'death'
            
        smooth: bool
            wether to smooth the data frame or not.
            The smoothing is done by using a 7-day rolling average   
    '''
    
    assert isinstance(smooth, bool), "Smooth must be a boolean variable!"
    
    base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
    
    
    if target == 'case':
        jh_data_url = base_url + 'csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
        cols_to_drp = ['UID', 'iso2', 'iso3', 'code3','Country_Region', 'Lat', 'Long_']

    elif target=='death':
        jh_data_url = base_url + 'csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
        cols_to_drp = ['UID', 'iso2', 'iso3', 'code3','Country_Region', 'Lat', 'Long_','Population']
    else:
        print("invalid argument for target. Acceptable values are: 'case' or 'death'")
        return None

    jh_covid_df = pd.read_csv(jh_data_url)

    # preprocessing JH COVID data
    jh_covid_df.dropna(axis=0, how='any', inplace=True)

    jh_covid_df['FIPS'] = jh_covid_df['FIPS'].astype('int64')

    jh_covid_df.drop(columns=cols_to_drp, inplace=True)

    #Important: check to see the column index is adherent to the imported df

    first_date = datetime.strptime(jh_covid_df.columns[4], '%m/%d/%y').date()

    last_date = datetime.strptime(jh_covid_df.columns[-1], '%m/%d/%y').date()


    current_date = last_date

    previous_date = last_date - timedelta (days=1)


    while current_date > first_date:

        #For unix, replace # with - in the time format

        current_col = current_date.strftime(conversion_format) #replace # with - in Mac or Linux

        previous_col = previous_date.strftime(conversion_format)

        jh_covid_df[previous_col] = np.where(jh_covid_df[previous_col] > jh_covid_df[current_col], 
                                             jh_covid_df[current_col], jh_covid_df[previous_col])

        current_date = current_date - timedelta(days=1)

        previous_date = previous_date - timedelta(days=1)
        
    
    if smooth:
        jh_covid_df.iloc[:,4:] = jh_covid_df.iloc[:,4:].rolling(7,min_periods=1,axis=1).mean()


    return jh_covid_df

In [ ]:
covid_df = get_JH_covid_data('case', True)

### Load Facebook Movement Range Data

In [ ]:
fb_mobility = pd.read_csv(input_directory + 'movement-range-2021-03-02.txt', sep="\t", dtype={'polygon_id':str})

In [ ]:
fb_mobility_us = fb_mobility[fb_mobility['country']=='USA']

In [ ]:
# Number of unique counties for which we have at least one day of data
len(fb_mobility_us['polygon_id'].unique())

In [ ]:
# counting Counties in the contiguous US for which there is no data in FB mobility
contiguous_fips = set(data['GEOID']) # number of unique fips: 3103
mobility_fips = set(fb_mobility_us['polygon_id']) # number of unique fips: 2694
i = 0
missing_fips = []
for fips in contiguous_fips:
    if (fips in mobility_fips):
        i+=1
    else:
        missing_fips.append(fips)

In [ ]:
# number of Counties in the contiguous US for which there is no data in FB mobility
len(missing_fips)

In [ ]:
# Create a new dataframe as transpose of the above, with days as columns and counties as rows

relative_df = pd.DataFrame(columns=fb_mobility_us['ds'].unique(), index=data['GEOID'].unique(), dtype=float)
ratio_df = pd.DataFrame(columns=fb_mobility_us['ds'].unique(), index=data['GEOID'].unique(), dtype=float)

In [ ]:
fb_mobility_us

In [ ]:
idx_contiguous = fb_mobility_us.index[fb_mobility_us['polygon_id'].isin(contiguous_fips)]

In [ ]:
fb_mobility_contiguous = fb_mobility_us.loc[idx_contiguous]

In [ ]:
import time
start = time.time()

for index, row in fb_mobility_contiguous.iterrows():
    relative_df.loc[row['polygon_id']][row['ds']] = row['all_day_bing_tiles_visited_relative_change']
    ratio_df.loc[row['polygon_id']][row['ds']] = row['all_day_ratio_single_tile_users']
    

end = time.time()
print(end - start)

In [ ]:
relative_df.head()

In [ ]:
print(relative_df.shape , ratio_df.shape)

### Impute FB mobility dataframes
The two dataframes above have a lot of Nan values which should be imputed by state average

In [ ]:
temp_ratio_df = data[['GEOID', 'STATEFP']].merge(ratio_df, left_on='GEOID', right_index=True, how='left')

In [ ]:
for col in temp_ratio_df.iloc[:,2:].columns:
    temp_ratio_df[col] = temp_ratio_df.groupby('STATEFP')[col].transform(lambda x: x.fillna(x.mean()))

In [ ]:
temp_ratio_df.isna().sum().sum()

In [ ]:
temp_relative_df = data[['GEOID', 'STATEFP']].merge(relative_df, left_on='GEOID', right_index=True, how='left')

In [ ]:
for col in temp_relative_df.iloc[:,2:].columns:
    temp_relative_df[col] = temp_relative_df.groupby('STATEFP')[col].transform(lambda x: x.fillna(x.mean()))

In [ ]:
temp_relative_df.isna().sum().sum()

In [ ]:
relative_df_smooth = temp_relative_df.copy()
ratio_df_smooth = temp_ratio_df.copy()

relative_df_smooth.iloc[:,2:] = relative_df_smooth.iloc[:,2:].rolling(7,min_periods=1, axis=1).mean()
ratio_df_smooth.iloc[:,2:] = ratio_df_smooth.iloc[:,2:].rolling(7,min_periods=1, axis=1).mean()

In [ ]:
relative_df_smooth.iloc[:,2:].columns

In [ ]:
relative_df_smooth.shape

## Add Social Proximity to Cases

In [ ]:
SCI_df = pd.read_csv(input_directory + 'SCI_matrix.csv', dtype={'Unnamed: 0':str})

In [ ]:
SCI_df.set_index('Unnamed: 0', inplace=True)

In [ ]:
# Create normalized SCI. It is calculated by dividing all the columns of the sci_matrix by the sum of the rpw
# This would give us the second term in social proximity formula above

sci_matrix_normal = SCI_df.div(SCI_df.sum(axis=1), axis=0)

In [ ]:
# set diagonal to zero
sci_matrix_normal.values[[np.arange(sci_matrix_normal.shape[0])]*2] = 0

**The matrix above is created for the entire US, but we are using contiguous US data here, therefore some rows and columns should be removed**

In [ ]:
to_drop=[]

for index in sci_matrix_normal.index:
    if not index in contiguous_fips:
        to_drop.append(index)

In [ ]:
sci_matrix_normal.drop(to_drop, inplace=True)

In [ ]:
sci_matrix_normal.drop(to_drop, axis=1, inplace=True)

In [ ]:
sci_matrix_normal.shape

## Add SafeGraph mobility features

Updated based on forecast hub dates

In [ ]:
safegraph_mobility = pd.read_csv(input_directory + 'safegraph_mobility.csv', dtype={'county_fips':str})

In [ ]:
safegraph_contiguous = safegraph_mobility[safegraph_mobility['county_fips'].isin(contiguous_fips)]

In [ ]:
len(safegraph_contiguous['county_fips'].unique())

In [ ]:
temp_cols = safegraph_contiguous.drop(['start_date', 'end_date', 'base_start', 'base_end'], axis=1)
safegraph_metrics = temp_cols.columns

In [ ]:
len(safegraph_metrics)

## Load Temperature Data

In [ ]:
max_temp = pd.read_csv(input_directory + 'max_temp_df_2021.csv', dtype={'GEOID':str})

In [ ]:
min_temp = pd.read_csv(input_directory + 'min_temp_df_2021.csv', dtype={'GEOID':str})

## Helper functions

In [ ]:
# return FCI-normal table for the input date
# set path_to_fci to where FCI matrices are stored
def get_normal_fci(date):
    path_to_fci = './output/' + str(date.year) + '/'+ date.strftime('%m') + 
                '/FCI_normal/' + date.strftime('%Y-%m-%d') + '-FCI-normal.csv'
    fci_norm = pd.read_csv(path_to_fci, dtype={'Unnamed: 0':str})
    fci_norm.set_index('Unnamed: 0', inplace=True)
    
    to_drop=[]

    for index in fci_norm.index:
        if not index in contiguous_fips:
            to_drop.append(index)
            
    fci_norm.drop(to_drop, inplace=True)
    fci_norm.drop(to_drop, axis=1, inplace=True)
    return fci_norm

In [ ]:
# calculates weekly average FPC using the end date and the start date of the week
# the input to this fuction should be of type datetime
def weekly_mean_FPC(end_date, start_date, logged=False):
    
    dates = [end_date]
    while end_date> start_date:
        end_date -= timedelta(days=1)
        dates.append(end_date)
    
    
    temp = data[['GEOID','FIPS', 'POPULATION']]
    
    
    for date in dates:
        # convert date to String
        date_str = date.strftime('%Y-%m-%d')
        
        temp = temp.merge(covid_df_contiguous[['FIPS', date_str]], on='FIPS', how='left')
        
        if logged:
            temp['inc_rate_' + date_str] = np.log(temp[date_str] / temp['POPULATION'] * 10000 + 1)
            
        else:
            temp['inc_rate_' + date_str] = temp[date_str] / temp['POPULATION'] * 10000
        
        normal_fci = get_normal_fci(date)
        normal_fci = normal_fci.merge(temp, left_index= True, right_on='GEOID')
        
        normal_fci['fpc_'+ date_str] = np.dot(normal_fci.iloc[:,:number_counties], normal_fci['inc_rate_' + date_str])
        
    
    normal_fci['mean_fpc'] = normal_fci.iloc[:,-len(dates):].mean(axis=1)
        
    return normal_fci[['GEOID','mean_fpc']]    

In [ ]:
# calculates weekly average SPC
# the input to this fuction should be of type datetime
def weekly_mean_SPC(end_date, start_date, logged=False):
    
    dates = [end_date]
    while end_date> start_date:
        end_date -= timedelta(days=1)
        dates.append(end_date)
    
    
    temp = data[['GEOID','FIPS', 'POPULATION']]
    
    for date in dates:
        # convert date to String
        date_str = date.strftime('%Y-%m-%d')
        
        temp = temp.merge(covid_df_contiguous[['FIPS', date_str]], on='FIPS', how='left')
        
        if logged:
            temp['inc_rate_' + date_str] = np.log(temp[date_str] / temp['POPULATION'] * 10000 + 1)
            
        else:
            temp['inc_rate_' + date_str] = temp[date_str] / temp['POPULATION'] * 10000
        
        
        normal_sci = sci_matrix_normal.merge(temp, left_index= True, right_on='GEOID')
        
        normal_sci['spc_'+ date_str] = np.dot(normal_sci.iloc[:,:number_counties], normal_sci['inc_rate_' + date_str])
        
    
    normal_sci['mean_spc'] = normal_sci.iloc[:,-len(dates):].mean(axis=1)
        
    return normal_sci[['GEOID','mean_spc']]

In [ ]:
# the input to this fuction should be of type datetime.
# returns a subset of FB movement range dfs based on the given week
def weekly_fb_mobility(end_date, start_date, df):
    
    dates = [end_date]
    while end_date> start_date:
        end_date -= timedelta(days=1)
        dates.append(end_date)
    
    dates_str=[]
    for date in dates:
        # convert date to String
        dates_str.append(date.strftime('%Y-%m-%d'))
    
    return df[['GEOID', *dates_str]]

In [ ]:
# calculate slope features

from scipy.stats import linregress

def linear_reg(week_df):
    
    x = np.arange(1,(week_df.shape[1]),1)
    x = (x - np.mean(x))/ np.std(x)
    
    slopes=[]
    
    for index, row in week_df.iloc[:,1:].iterrows():
        y = row
        y = (y - np.mean(y))/ np.std(y)
        slopes.append(linregress(x, y)[0])
        
    week_df.loc[:,'slope'] = slopes
    return week_df[['GEOID','slope']]

## Final function to combine all features generated above

This function generates a dataframe and for a given date, will add the following features to the dataframe

- incidence rate data
- FB mobility data (ratio, relative)
- SPC (facebook SCI and incidence rates)
- SafeGraph mobility 
- FPC (FCI and incidence rate)

For each period, there is a 5 week difference between the actual date (t) and the start of the second lag. For example if `T: Oct 1 (Sep 24 to Oct 1)`, then `T-1: Sep 10 to Sep 24`, and `T-2: August 27 to Sep 10`.

Since the earliest day for which we have FB mobility data is March 1, the rearliest  (end) date for T will be April 5th.

In [ ]:
covid_df_contiguous = data[['FIPS','STATEFP','COUNTYFP','GEOID']].merge(covid_df, on='FIPS', how='left')

In [ ]:
# Main function
# the input to the funtion is the end date for which we want to add data

from datetime import datetime, timedelta


def add_lagged_data(date):
    global data
    
    T_end = date
    T_start = T_end - timedelta(days=6)
    
    T_1_end = T_start - timedelta(days=1)
    T_1_start = T_1_end - timedelta(days=6)
    
    T_2_end = T_1_start - timedelta(days=1)
    T_2_start = T_2_end - timedelta(days=6)
    
    T_3_end = T_2_start - timedelta(days=1)
    T_3_start = T_3_end - timedelta(days=6)
    
    T_4_end = T_3_start - timedelta(days=1)
    T_4_start = T_4_end - timedelta(days=6)
    
    # These dates are used for cumulative cases (Saturday to Saturday)
    T_start_case = T_end - timedelta(days=7)
    T_1_start_case = T_1_end - timedelta(days=7)
    T_2_start_case = T_2_end - timedelta(days=7)
    T_3_start_case = T_3_end - timedelta(days=7)
    T_4_start_case = T_4_end - timedelta(days=7)
    

    dates = [T_end.strftime('%Y-%m-%d'), T_start.strftime('%Y-%m-%d'),
             T_1_end.strftime('%Y-%m-%d'), T_1_start.strftime('%Y-%m-%d'), 
             T_2_end.strftime('%Y-%m-%d'), T_2_start.strftime('%Y-%m-%d'),
             T_3_end.strftime('%Y-%m-%d'), T_3_start.strftime('%Y-%m-%d'), 
             T_4_end.strftime('%Y-%m-%d'), T_4_start.strftime('%Y-%m-%d')]
    
    
    dates_non_str = [T_end, T_start,
             T_1_end, T_1_start, 
             T_2_end, T_2_start,
             T_3_end, T_3_start, 
             T_4_end, T_4_start]
    
    dates_case = [T_end.strftime('%Y-%m-%d'), T_start_case.strftime('%Y-%m-%d'),
             T_1_end.strftime('%Y-%m-%d'), T_1_start_case.strftime('%Y-%m-%d'), 
             T_2_end.strftime('%Y-%m-%d'), T_2_start_case.strftime('%Y-%m-%d'),
             T_3_end.strftime('%Y-%m-%d'), T_3_start_case.strftime('%Y-%m-%d'), 
             T_4_end.strftime('%Y-%m-%d'), T_4_start_case.strftime('%Y-%m-%d')]
    
    
    dates_non_str_case = [T_end, T_start_case,
             T_1_end, T_1_start_case, 
             T_2_end, T_2_start_case,
             T_3_end, T_3_start_case, 
             T_4_end, T_4_start_case]

    
    temp = data.copy()
    
    temp['date_end_period'] = T_end.strftime('%Y-%m-%d')
    temp['date_start_period'] = T_start.strftime('%Y-%m-%d')
    temp['date_end_lag'] = T_1_end.strftime('%Y-%m-%d')
    temp['date_start_lag'] = T_4_start.strftime('%Y-%m-%d')
    
    time_periods = ['T_end', 'T_start', 'T_1_end', 'T_1_start','T_2_end','T_2_start',
                    'T_3_end','T_3_start','T_4_end','T_4_start']
    i = 0
    for period in time_periods:
        
        
        temp = temp.merge(covid_df_contiguous[['GEOID',dates_case[i]]], on='GEOID', how='left')
        temp.rename(columns={dates_case[i]:'case_'+ period}, inplace=True)

        
        temp['inc_rate_' + period] = temp['case_'+ period] / temp['POPULATION'] * 10000
        
        temp = temp.merge(relative_df_smooth[['GEOID',dates[i]]], on='GEOID', how='left')
        temp.rename(columns={dates[i]:'relative_'+ period}, inplace=True) 


        temp = temp.merge(ratio_df_smooth[['GEOID',dates[i]]], on='GEOID', how='left')
        temp.rename(columns={dates[i]:'ratio_'+ period}, inplace=True)
        
        # The same date is used as the input to weekly_mean_SPC function to calculate
        # SPC for that given date (instead of an average over a period)
        temp = temp.merge(weekly_mean_SPC(dates_non_str_case[i],dates_non_str_case[i], logged=False), on='GEOID', how='left')
        temp.rename(columns={'mean_spc':'SPC_'+ period}, inplace=True)

        # simiar to SPC, add FPC values
        temp = temp.merge(weekly_mean_FPC(dates_non_str_case[i],dates_non_str_case[i], logged=False), on='GEOID', how='left')
        temp.rename(columns={'mean_fpc':'FPC_'+ period}, inplace=True)
        
        
        # add logged SPC (defined as log(delta incidence rate)*sci/sum(sci))
        temp = temp.merge(weekly_mean_SPC(dates_non_str_case[i],dates_non_str_case[i], logged=True), on='GEOID', how='left')
        temp.rename(columns={'mean_spc':'SPC_logged_'+ period}, inplace=True)
        
        
        # add logged FPC (defined as log(delta incidence rate)*fci/sum(fci))
        temp = temp.merge(weekly_mean_FPC(dates_non_str_case[i],dates_non_str_case[i], logged=True), on='GEOID', how='left')
        temp.rename(columns={'mean_fpc':'FPC_logged_'+ period}, inplace=True)

        # add raw John Hopkins case data
        temp = temp.merge(jh_covid_df[['FIPS',dates_non_str_case[i].strftime('%#m/%#d/%y')]], on='FIPS', how='left')
        temp.rename(columns={dates_non_str_case[i].strftime('%#m/%#d/%y'):'case_JH_'+ period}, inplace=True)
        
        # add smoothed John Hopkins case data
        temp = temp.merge(covid_df_contiguous[['FIPS',dates_case[i]]], on='FIPS', how='left')
        temp.rename(columns={dates_case[i]:'case_JH_smoothed_'+ period}, inplace=True)
       
        
        
        i+=1
    
    times = ['T', 'T_1', 'T_2', 'T_3', 'T_4']
    
    j = 0
    for period in times:
        temp['DELTA_INC_RATE_' + period] = temp['inc_rate_' + time_periods[j]] - temp['inc_rate_' + time_periods[j+1]]
        
        temp['DELTA_REL_MOB_' + period] = temp['relative_' + time_periods[j]] - temp['relative_' + time_periods[j+1]]
        temp['DELTA_RATIO_MOB_' + period] = temp['ratio_' + time_periods[j]] - temp['ratio_' + time_periods[j+1]]
        
        temp['DELTA_SPC_' + period] = temp['SPC_' + time_periods[j]] - temp['SPC_' + time_periods[j+1]]
        temp['DELTA_FPC_' + period] = temp['FPC_' + time_periods[j]] - temp['FPC_' + time_periods[j+1]]
        temp['DELTA_SPC_LOGGED_' + period] = temp['SPC_logged_' + time_periods[j]] - temp['SPC_logged_' + time_periods[j+1]]
        temp['DELTA_FPC_LOGGED_' + period] = temp['FPC_logged_' + time_periods[j]] - temp['FPC_logged_' + time_periods[j+1]]
        
        temp['DELTA_CASE_JH_' + period] = temp['case_JH_'+ time_periods[j]] - temp['case_JH_'+ time_periods[j+1]]
        temp['DELTA_CASE_JH_SMOOTH_' + period] = temp['case_JH_smoothed_'+ time_periods[j]] - 
                                                 temp['case_JH_smoothed_'+ time_periods[j+1]]
        
        # mean incidence rate is calculated between Sunday and Saturday
        temp['MEAN_INC_RATE_' + period] = covid_df_contiguous[pd.date_range(start=dates[j+1], end=dates[j])
                                            .strftime('%Y-%m-%d')].mean(axis=1) / temp['POPULATION'] * 10000
        temp['MEAN_REL_MOB_' + period] = relative_df_smooth[pd.date_range(start=dates[j+1], end=dates[j])
                                            .strftime('%Y-%m-%d')].mean(axis=1)
        temp['MEAN_RATIO_MOB_' + period] = ratio_df_smooth[pd.date_range(start=dates[j+1], end=dates[j])
                                            .strftime('%Y-%m-%d')].mean(axis=1)

        
        # add Safegraph mobility features
        safegraph_data = safegraph_contiguous[safegraph_contiguous['end_date']==dates[j]][safegraph_metrics]
        temp = temp.merge(safegraph_data, left_on='GEOID', right_on='county_fips')
        
        rename_dict = dict()
        for col in safegraph_metrics[1:]:
            rename_dict[col] = col + '_' + period
            
        temp.rename(columns=rename_dict, inplace=True)
        
        
        # add MEAN_FPC
        temp = temp.merge(weekly_mean_FPC(dates_non_str_case[j], dates_non_str_case[j+1]), on='GEOID', how='left')
        temp.rename(columns={'mean_fpc':'MEAN_FPC_'+ period}, inplace=True)
        
        # add logged MEAN_FPC 
        temp = temp.merge(weekly_mean_FPC(dates_non_str_case[j], dates_non_str_case[j+1], logged=True), on='GEOID', how='left')
        temp.rename(columns={'mean_fpc':'MEAN_FPC_LOGGED_'+ period}, inplace=True)
        
        # add MEAN_SPC
        temp = temp.merge(weekly_mean_SPC(dates_non_str_case[j], dates_non_str_case[j+1]), on='GEOID', how='left')
        temp.rename(columns={'mean_spc':'MEAN_SPC_'+ period}, inplace=True)
        
        # add logged MEAN_SPC
        temp = temp.merge(weekly_mean_SPC(dates_non_str_case[j], dates_non_str_case[j+1], logged=True), on='GEOID', how='left')
        temp.rename(columns={'mean_spc':'MEAN_SPC_LOGGED_'+ period}, inplace=True)
        
        # add FB mobility slopes
        ratio_slope = linear_reg(weekly_fb_mobility(dates_non_str[j], dates_non_str[j+1], ratio_df_smooth))
        temp = temp.merge(ratio_slope, on='GEOID', how='left')
        temp.rename(columns={'slope':'SLOPE_RATIO_MOB_'+ period}, inplace=True)
        
        relative_slope = linear_reg(weekly_fb_mobility(dates_non_str[j], dates_non_str[j+1], relative_df_smooth))
        temp = temp.merge(relative_slope, on='GEOID', how='left')
        temp.rename(columns={'slope':'SLOPE_REL_MOB_'+ period}, inplace=True)

        
        
        # add temperature features
        # to update for the new dates, min and max temperature are used with one day offset
        adj_temp_date = (dates_non_str[j] + timedelta(days=1)).strftime('%Y-%m-%d')
        
        temp = temp.merge(max_temp[['GEOID', adj_temp_date]], on='GEOID', how='left')
        temp.rename(columns={adj_temp_date:'MAX_TEMP_'+ period}, inplace=True)
        
        temp = temp.merge(min_temp[['GEOID', adj_temp_date]], on='GEOID', how='left')
        temp.rename(columns={adj_temp_date:'MIN_TEMP_'+ period}, inplace=True)

        j += 2

    output_df = temp.copy()
    
    return output_df

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
end_date = datetime(2021, 2, 27)
week_counter = 0
df_list = []

while end_date > datetime(2020, 3, 29):
    
    start_time = time.time()
    
    df_list.append(add_lagged_data(end_date))
    
    end_time = time.time()
    print('Feature generation for the week ending in {} finished in {} seconds'.format(end_date.strftime('%Y-%m-%d'),
                                                                                       round(end_time-start_time,1)))
    
    end_date -= timedelta(weeks=1)
    week_counter+=1

In [ ]:
final_df = pd.concat(df_list, axis=0)

In [ ]:
# check number of weeks for which we have features
final_df.shape[0]/3103

### Imputing NAs

In [ ]:
# pd.set_option('max_rows', 200)
final_df.isna().sum().sum()

In [ ]:
pd.set_option('max_rows', 400)
final_df.isna().sum()

In [ ]:
data_to_save = final_df.copy()

In [ ]:
na_columns = data_to_save.columns[data_to_save.isnull().any()]

In [ ]:
# impute missing values by state average
for col in data_to_save[na_columns].columns:
    data_to_save[col] = data_to_save.groupby('STATEFP')[col].transform(lambda x: x.fillna(x.mean()))

In [ ]:
data_to_save.isna().sum().sum()

In [ ]:
times = ['T', 'T_1', 'T_2', 'T_3', 'T_4']

for period in times:
    data_to_save['LOG_DELTA_INC_RATE_' + period] = np.log(data_to_save['DELTA_INC_RATE_' + period] + 1)
    data_to_save['LOG_MEAN_INC_RATE_' + period] = np.log(data_to_save['MEAN_INC_RATE_' + period] + 1)
    data_to_save['LOG_DELTA_SPC_' + period] = np.log(data_to_save['DELTA_SPC_' + period] + 1)
    data_to_save['LOG_DELTA_FPC_' + period] = np.log(data_to_save['DELTA_FPC_' + period] + 1)

In [ ]:
final_cols = [
'GEOID',
'NAME',
'State_Name',
'STATEFP', 
'COUNTYFP', 
'date_end_period',
'date_start_period',
'date_end_lag',
'date_start_lag',
'LOG_DELTA_INC_RATE_T',
'PCT_MALE',
'PCT_BLACK',
'PCT_HISPAN', 
'PCT_AMIND',
'PCT_RURAL',
'PCT_COL_DE' ,
'PCT_TRUMP_',
'MED_HOS_IN',
'POPULATION',
'DELTA_CASE_JH_T',
'DELTA_CASE_JH_SMOOTH_T'
]

In [ ]:
additional_cols=[
'LOG_DELTA_INC_RATE_T_',
'DELTA_REL_MOB_T_',
'DELTA_RATIO_MOB_T_',
'DELTA_SPC_T_',
'DELTA_SPC_LOGGED_T_',
'DELTA_FPC_T_',
'DELTA_FPC_LOGGED_T_',
'LOG_MEAN_INC_RATE_T_',
'MEAN_REL_MOB_T_',
'MEAN_RATIO_MOB_T_',
'MEAN_FPC_T_',
'MEAN_SPC_T_',
'SLOPE_RATIO_MOB_T_',
'SLOPE_REL_MOB_T_',
'MAX_TEMP_T_',
'MIN_TEMP_T_',
'pct_completely_home_device_count_current_T_',
'pct_full_time_work_behavior_devices_current_T_',
'pct_part_time_work_behavior_devices_current_T_',
'pct_delivery_behavior_devices_current_T_',
'distance_traveled_from_home_current_T_',
'median_home_dwell_time_current_T_',
'pct_completely_home_device_count_baselined_T_',
'pct_full_time_work_behavior_devices_baselined_T_',
'pct_part_time_work_behavior_devices_baselined_T_',
'pct_delivery_behavior_devices_baselined_T_',
'distance_traveled_from_home_baselined_T_',
'median_home_dwell_time_baselined_T_',
'pct_completely_home_device_count_slope_T_',
'pct_full_time_work_behavior_devices_slope_T_',
'pct_part_time_work_behavior_devices_slope_T_',
'pct_delivery_behavior_devices_slope_T_',
'distance_traveled_from_home_slope_T_',
'median_home_dwell_time_slope_T_',
'DELTA_CASE_JH_T_',
'MEAN_SPC_LOGGED_T_',
'MEAN_FPC_LOGGED_T_'
]

for i in range(1,5):
    for col in additional_cols:
        final_cols.append(col+str(i))
    

In [ ]:
output = data_to_save[final_cols]

In [ ]:
output.to_csv('./output/all_features_updated_incidence.csv')

### Creating dataframes for 2, 3, and 4-week predictions

in this dataframe, the target variables is the the number of cumulative cases in 2, 3, and 4 weeks ahead, denoted by `LOG_DELTA_INC_RATE_T_14`, `LOG_DELTA_INC_RATE_T_21`, and `LOG_DELTA_INC_RATE_T_28` respectively.

In [ ]:
# Main function
# the input to the funtion is the end date for which we want to add data

from datetime import datetime, timedelta


def add_lagged_y(date):
    global output, jh_covid_df
    
    T_end = date
    T_start = T_end - timedelta(days=7)
    
    T_start_period = (T_end - timedelta(days=6)).strftime('%Y-%m-%d')
    
    T_14 =  T_end + timedelta(days=7)
    T_21 =  T_end + timedelta(days=14)
    T_28 =  T_end + timedelta(days=21)
    

    dates_non_str = [T_end, T_start, T_14, T_21, T_28]
    
    dates = [item.strftime('%Y-%m-%d') for item in dates_non_str]
    
    dates_jh = [item.strftime('%#m/%#d/%y') for item in dates_non_str]
    
    
    periods = ['T_end', 'T_start', 'T_14', 'T_21', 'T_28']
    
    temp = output.loc[(output.date_end_period==dates[0]) & (output.date_start_period==T_start_period)].copy()
    
    temp['FIPS'] = temp['GEOID'].astype(int)
    
    #print('check 1 {}'.format(temp.shape))
    temp['target_date_2wk'] = T_14.strftime('%Y-%m-%d')
    temp['target_date_3wk'] = T_21.strftime('%Y-%m-%d')
    temp['target_date_4wk'] = T_28.strftime('%Y-%m-%d')
        
    temp = temp.merge(covid_df_contiguous[['GEOID',*dates]], on='GEOID', how='left')
    
    temp = temp.merge(jh_covid_df[['FIPS',*dates_jh]], on='FIPS', how='left')
    #print('check 2 {}'.format(temp.shape))

    for period, date in zip(periods, dates):
        temp['inc_rate_' + period] = temp[date] / temp['POPULATION'] * 10000


    for period, date in zip(periods[-3:], dates[-3:]):
        temp['DELTA_CASE_SMOOTHED_' + period] = temp[date] - temp[dates[1]]
        temp['DELTA_INC_RATE_' + period] = temp['inc_rate_' + period] - temp['inc_rate_T_start']
        temp['LOG_DELTA_INC_RATE_' + period] = np.log(temp['DELTA_INC_RATE_' + period] + 1)
    
    for period, date in zip(periods[-3:], dates_jh[-3:]):
        temp['DELTA_CASE_JH_' + period] = temp[date] - temp[dates_jh[1]]
    
    temp['DELTA_CASE_JH_T'] = temp[dates_jh[0]] - temp[dates_jh[1]]
        
    
    
    cols = ['target_date_2wk','LOG_DELTA_INC_RATE_T_14', 
            'target_date_3wk','LOG_DELTA_INC_RATE_T_21',
            'target_date_4wk','LOG_DELTA_INC_RATE_T_28' ]
    #print('check 3 {}'.format(temp.shape))
    
    return temp[[*output.columns,'DELTA_CASE_JH_T',
            'target_date_2wk','LOG_DELTA_INC_RATE_T_14', 'DELTA_CASE_SMOOTHED_T_14', 'DELTA_CASE_JH_T_14',
            'target_date_3wk','LOG_DELTA_INC_RATE_T_21', 'DELTA_CASE_SMOOTHED_T_21', 'DELTA_CASE_JH_T_21',
            'target_date_4wk','LOG_DELTA_INC_RATE_T_28', 'DELTA_CASE_SMOOTHED_T_28', 'DELTA_CASE_JH_T_28']]
    
    
   

In [ ]:
end_date = datetime(2021, 2, 27)
df_lagged_list = []

while end_date > datetime(2020, 3, 29):
    
    start_time = time.time()
    
    df_lagged_list.append(add_lagged_y(end_date))
    
    end_time = time.time()
    print('Feature generation for the week ending in {} finished in {} seconds'.format(end_date.strftime('%Y-%m-%d'),
                                                                                       round(end_time-start_time,1)))
    end_date -= timedelta(weeks=1)

In [ ]:
df_lagged = pd.concat(df_lagged_list, axis=0)

In [ ]:
df_lagged.isna().sum().sum()

In [ ]:
df_lagged.shape, df_lagged.shape[0]/3103

In [ ]:
df_lagged.to_csv('./output/all_features.csv')